<a href="https://colab.research.google.com/github/ThuanyDeSouza/N3_Security/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
# Gerando as chaves
import linecache
import random
import math

# Função para gerar um número primo grande aleatório.
def generate_large_prime(file_path):
    line_count = sum(1 for line in open(file_path))
    random_line_numbers = random.sample(range(1, line_count + 1), 2)
    random_lines = [linecache.getline(file_path, line_number).strip() for line_number in random_line_numbers]
    return random_lines

# Função para verificar se um número é primo.
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

# Função para calcular o maior divisor comum entre dois números.
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#  Função para calcular o inverso modular de 'a' mod 'm'.
def mod_inverse(a, m):
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (
            u1 - q * v1,
            u2 - q * v2,
            u3 - q * v3,
            v1,
            v2,
            v3,
        )

    return u1 % m

# Função para gerar as chaves RSA.
def generate_rsa_keys():
    p, q = generate_large_prime("primeList.txt")
    p = int(p)
    q = int(q)
    n = p * q
    totient = (p - 1) * (q - 1)

    while True:
        e = random.randint(2, totient)
        if gcd(e, totient) == 1:
            break

    d = mod_inverse(e, totient)

    return (n, e), (n, d)

# Exemplo de uso
public_key, private_key = generate_rsa_keys()
print("Chave pública:", public_key)
print("Chave privada:", private_key)
# Escreve o arquivo encriptado
with open("public.txt", 'w') as file:
    file.write('\n'.join(str(x) for x in public_key))

# Escreve o arquivo encriptado
with open("private.txt", 'w') as file:
    file.write('\n'.join(str(x) for x in private_key))


Chave pública: (15212891271636492509, 14976617133418967297)
Chave privada: (15212891271636492509, 8448873144947601593)


In [102]:
# Encripta

import base64

def encrypt(public_key, message):
    n, e = public_key
    encrypted = []
    for msg in message:
       encodedChunk = pow(msg, int(e), int(n))
       encrypted.append(encodedChunk)

    return encrypted


def read_keyfile(keyfile):
    with open(keyfile, 'r') as file:
        lines = []
        for line in file:
          lines.append(line)

        return lines

def chunk_base64_to_int(base64_string, chunk_size):
    base64_bytes = base64_string.encode('utf-8')
    decoded_bytes = base64.b64decode(base64_bytes)
    chunks = [decoded_bytes[i:i+chunk_size] for i in range(0, len(decoded_bytes), chunk_size)]
    int_chunks = [int.from_bytes(chunk, 'big') for chunk in chunks]
    return int_chunks

def blockSize(n):
    blockSize = 0
    temp = n - 1
    while temp > 1:
        temp = temp / 2
        blockSize += 1

    return blockSize / 8


def encrypt_file(keyfile, inputfile, outputfile):
    # Lê o arquivo a ser encriptado
    with open(inputfile, 'r') as file:
        data = file.read()

    # Converte o texto para base64
    data = base64.b64encode(data.encode()).decode()
    print("Base64: \n")
    print(data)
    print("\n")

    # Lê a chave
    public_key = read_keyfile(keyfile)
    print("Chave: \n")
    print(public_key)
    print("\n")

    # Retorna blocksize de acordo com n
    bs = blockSize(int(public_key[0]))
    print("BlockSize: \n")
    print(bs)
    print("\n")

    # Divide o texto de acordo com n e Converte para números longos
    test = chunk_base64_to_int(data, int(bs))
    print("Texto dividido e convertido: \n")
    print(test)
    print("\n")

    # Encriptar dados
    encrypted = encrypt(public_key, test)
    print("Encriptado: \n")
    print(encrypted)
    print("\n")
    y = 0
    for x in encrypted:
      y +=1

    print(y)

    # Escreve o arquivo encriptado
    with open(outputfile, 'w') as file:
        file.write('\n'.join(str(x) for x in encrypted))


if __name__ == '__main__':
    keyfile = 'public.txt'
    inputfile = 'input.txt'
    outputfile = 'output.txt'

    encrypt_file(keyfile, inputfile, outputfile)


Base64: 

TG9yZW0gaXBzdW0gZG9sb3Igc2l0IGFtZXQsIGNvbnNlY3RldHVyIGFkaXBpc2NpbmcgZWxpdC4KRG9uZWMgYXVndWUgYXVndWUsIHBlbGxlbnRlc3F1ZSBhdCBtb2xsaXMgbm9uLCBldWlzbW9kIGFjIGV4LgpOdWxsYSBzb2xsaWNpdHVkaW4gY29uZ3VlIHVybmEsIGVnZXQgdWxsYW1jb3JwZXIgbG9yZW0gY29udmFsbGlzIHNpdCBhbWV0LgpWaXZhbXVzIHNjZWxlcmlzcXVlIHF1aXMgbGFjdXMgc2VkIHJob25jdXMuClByb2luIGNvbW1vZG8gZWxpdCBlZ2V0IGF1Y3RvciBsb2JvcnRpcy4KU2VkIHZ1bHB1dGF0ZSBkdWkgYSB2ZWxpdCBtb2xlc3RpZSwgdmVsIGxvYm9ydGlzIG9yY2kgdGVtcG9yLgpQZWxsZW50ZXNxdWUgc2FnaXR0aXMgZXJhdCBzaXQgYW1ldCBhbGlxdWFtIGN1cnN1cy4gQ3JhcyBzZWQgZXN0IG51bGxhLgpQcm9pbiB2aXRhZSBjb252YWxsaXMgdXJuYS4gQWVuZWFuIG1hdXJpcyBtZXR1cywgcGxhY2VyYXQgcmhvbmN1cyBkdWkgdXQsIGRhcGlidXMgYXVjdG9yIHVybmEu


Chave: 

['15212891271636492509\n', '14976617133418967297']


BlockSize: 

8.0


Texto dividido e convertido: 

[5507746649245510000, 8319675872528264303, 8223699816335565165, 7310516612747980403, 7305810999619449376, 7017850050876498793, 7955362886755906606, 739838758733684833, 8459859502234170727, 8459215992

In [103]:
# Decripta

import base64

def decrypt(private_key, message):
    n, d = private_key
    decrypted = []
    for msg in message:
       encodedChunk = pow(int(msg), int(d), int(n))
       decrypted.append(encodedChunk)

    return decrypted


def read_keyfile(keyfile):
    with open(keyfile, 'r') as file:
        lines = []
        for line in file:
          lines.append(line)

        return lines

def chunk_int_to_base64(int_chunks, chunk_size):
    bytes_chunks = [int.to_bytes(chunk, length=(chunk.bit_length()+7)//8, byteorder='big') for chunk in int_chunks]
    decoded_bytes = b''.join(bytes_chunks)
    base64_bytes = base64.b64encode(decoded_bytes)
    base64_string = base64_bytes.decode('utf-8')
    return base64_string

def blockSize(n):
    blockSize = 0
    temp = n - 1
    while temp > 1:
        temp = temp / 2
        blockSize += 1

    return blockSize / 8


def decrypt_file(keyfile, inputfile, outputfile):
    # Lê o arquivo a ser descriptografado
    with open(inputfile, 'r') as file:
        lines = []
        for line in file:
          lines.append(line)

    # Lê a chave
    private_key = read_keyfile(keyfile)
    print("Chave: \n")
    print(private_key)
    print("\n")

    # Deseencriptar dados
    decrypted = decrypt(private_key, lines)
    print("Deseencriptado: \n")
    print(decrypted)
    print("\n")

    # Retorna blocksize de acordo com n
    bs = blockSize(int(private_key[0]))
    print("BlockSize: \n")
    print(bs)
    print("\n")

    # Junta o texto e Converte para base64
    test = chunk_int_to_base64(decrypted, int(bs))
    print("Texto dividido e convertido: \n")
    print(test)
    print("\n")

    # Converte o base64 para texto
    decoded_bytes = base64.b64decode(test)
    decoded_string = decoded_bytes.decode('utf-8')
    print("Base64: \n")
    print(decoded_string)
    print("\n")

    # Escreve o arquivo encriptado
    with open(outputfile, 'w') as file:
        file.write(decoded_string)


# Example usage
keyfile = 'private.txt'
inputfile = 'output.txt'
outputfile = 'newOutput.txt'

decrypt_file(keyfile, inputfile, outputfile)

Chave: 

['15212891271636492509\n', '8448873144947601593']


Deseencriptado: 

[5507746649245510000, 8319675872528264303, 8223699816335565165, 7310516612747980403, 7305810999619449376, 7017850050876498793, 7955362886755906606, 739838758733684833, 8459859502234170727, 8459215992921615468, 7308907224346228069, 2333273764578421868, 7598452654992141344, 7310865514584040480, 7017488264553433678, 8461256964334382956, 7811884333227402345, 7935451873671607653, 2338901399891091557, 7450488841312169057, 7882266310131151392, 7813589658459202415, 7959656509727142688, 8316305869338666356, 3317559086469442933, 8295757384357012850, 7598541745325896053, 7598452646166623603, 2338324112250923119, 7954330504420544626, 8028068896150875501, 8026662620812306804, 2334385667299107189, 7166475443014233954, 8030609081807800915, 7306000188650057845, 8386111958824547689, 2333181774834461044, 2336646293024371817, 7290237589685805164, 8026100023297667872, 8030590388722230637, 8101819796729914732, 781077057494131954